In [1]:
# 导入
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
plt.rcParams['font.sans-serif'] = ['SimHei']  # 设置中文字体为黑体
plt.rcParams['axes.unicode_minus'] = False  # 正确显示负号
import warnings
warnings.filterwarnings("ignore")
import xgboost as xgb
from xgboost import XGBClassifier

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import multilabel_confusion_matrix

In [3]:
# 读取数据
# df = pd.read_excel("222.xlsx")
df = pd.read_csv("M101.csv", index_col=0) #M101  test
df.head()

,时间,生产线编号,物料推送气缸推送状态,物料推送气缸收回状态,物料推送数,物料待抓取数,放置容器数,容器上传检测数,填装检测数,填装定位器固定状态,...,不合格数,物料推送装置故障1001,物料检测装置故障2001,填装装置检测故障4001,填装装置定位故障4002,填装装置填装故障4003,加盖装置定位故障5001,加盖装置加盖故障5002,拧盖装置定位故障6001,拧盖装置拧盖故障6002
日期,,,,,,,,,,,,,,,,,,,,,
1,0,M101,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,M101,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,M101,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,3,M101,1,0,2,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,4,M101,0,1,2,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
df = df.drop(['生产线编号'], axis=1)
df.set_index('时间', inplace=True)

# 建模
过采样或者使用原数据

In [5]:
df.columns

Index(['物料推送气缸推送状态', '物料推送气缸收回状态', '物料推送数', '物料待抓取数', '放置容器数', '容器上传检测数',
       '填装检测数', '填装定位器固定状态', '填装定位器放开状态', '物料抓取数', '填装旋转数', '填装下降数', '填装数',
       '加盖检测数', '加盖定位数', '推盖数', '加盖下降数', '加盖数', '拧盖检测数', '拧盖定位数', '拧盖下降数',
       '拧盖旋转数', '拧盖数', '合格数', '不合格数', '物料推送装置故障1001', '物料检测装置故障2001',
       '填装装置检测故障4001', '填装装置定位故障4002', '填装装置填装故障4003', '加盖装置定位故障5001',
       '加盖装置加盖故障5002', '拧盖装置定位故障6001', '拧盖装置拧盖故障6002'],
      dtype='object')

In [6]:
df

,物料推送气缸推送状态,物料推送气缸收回状态,物料推送数,物料待抓取数,放置容器数,容器上传检测数,填装检测数,填装定位器固定状态,填装定位器放开状态,物料抓取数,...,不合格数,物料推送装置故障1001,物料检测装置故障2001,填装装置检测故障4001,填装装置定位故障4002,填装装置填装故障4003,加盖装置定位故障5001,加盖装置加盖故障5002,拧盖装置定位故障6001,拧盖装置拧盖故障6002
时间,,,,,,,,,,,,,,,,,,,,,
0,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,2,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,2,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29005,0,1,4143,4143,1401,1400,1380,0,1,4140,...,18,0,0,0,0,0,0,0,0,0
29006,0,1,4143,4143,1401,1400,1380,0,1,4140,...,18,0,0,0,0,0,0,0,0,0
29007,0,1,4143,4143,1401,1400,1381,1,0,4141,...,18,0,0,0,0,0,0,0,0,0


In [7]:
n = len(df.columns)

# 选择倒数前九列
dfx = df.iloc[:, :n-9]

# df_trimmed 现在包含了 df 的倒数前九列的内容
dfx

,物料推送气缸推送状态,物料推送气缸收回状态,物料推送数,物料待抓取数,放置容器数,容器上传检测数,填装检测数,填装定位器固定状态,填装定位器放开状态,物料抓取数,...,推盖数,加盖下降数,加盖数,拧盖检测数,拧盖定位数,拧盖下降数,拧盖旋转数,拧盖数,合格数,不合格数
时间,,,,,,,,,,,,,,,,,,,,,
0,0,1,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,2,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,2,1,1,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29005,0,1,4143,4143,1401,1400,1380,0,1,4140,...,1398,1397,1397,1397,1397,1397,1397,1397,1378,18
29006,0,1,4143,4143,1401,1400,1380,0,1,4140,...,1398,1398,1398,1397,1397,1397,1397,1397,1378,18
29007,0,1,4143,4143,1401,1400,1381,1,0,4141,...,1398,1398,1398,1397,1397,1397,1397,1397,1378,18


In [8]:
# 将特定列中所有不等于 0 的值替换为 1
df['填装装置填装故障4003'] = (df['填装装置填装故障4003'] != 0).astype(int)
df['填装装置填装故障4003'].value_counts()

填装装置填装故障4003
0    633797
1      2712
Name: count, dtype: int64

In [9]:
X=dfx
y=df['填装装置填装故障4003']
# 划分训练集和测试集

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

In [10]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

# 假设df是你的DataFrame，并且已经按时间排序
# 选择输入特征和目标变量
features = dfx  # 这里假设'合格数'是目标变量
target = df['填装装置填装故障4003']

# 归一化特征
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_features = scaler.fit_transform(features)

# 划分数据窗口
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

time_steps = 10  # 例如，使用5个时间点的数据来预测下一个时间点
X, y = create_dataset(pd.DataFrame(scaled_features), target, time_steps)


In [11]:
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Input

# 假设你已经预处理了数据，并将其分为输入(X)和输出(y)

# 定义模型
model2 = Sequential()
model2.add(Input(shape=(X.shape[1], X.shape[2])))  # 添加Input层来定义输入形状
model2.add(LSTM(50, activation='relu'))  # 移除input_shape参数
model2.add(Dense(1))
model2.compile(optimizer='adam', loss='mse')

# 训练模型
model2.fit(X, y, epochs=100, verbose=0)

from sklearn.metrics import classification_report, confusion_matrix

# 使用模型进行预测
y_pred = model2.predict(X)
y_pred = (y_pred > 0.5)  # 根据需要调整阈值

# 计算混淆矩阵
cm = confusion_matrix(y, np.round(y_pred))
print('混淆矩阵：')
print(cm)

# 计算召回率等指标
report = classification_report(y, np.round(y_pred), target_names=['Class 0', 'Class 1'])
print('分类报告：')
print(report)


KeyboardInterrupt: 